In [1]:
import numpy as np
import scipy.ndimage as nd
import tensorflow as tf
import matplotlib.pyplot as plt

c:\Users\neera\Desktop\Lucid_demo\.venv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\neera\Desktop\Lucid_demo\.venv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\neera\Desktop\Lucid_demo\.venv\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\neera\Desktop\Lucid_demo\.venv\lib\site-packages\ten

In [3]:
import cv2, numpy as np, os

#parameters
working_dir = r'C:\Users\neera\Desktop\IIST\Assignments\CV project\shapes'
os.chdir(working_dir)
img_size = 60 #size of image fed into model

In [4]:
folders, labels, images = ['square', 'triangle'], [], []
for folder in folders:
 #   print folder
    for path in os.listdir(os.getcwd()+'/'+folder):
        img = cv2.imread(folder+'/'+path,0)
        #cv2.imshow('img', img)
        #cv2.waitKey(1)
        images.append(cv2.resize(img, (img_size, img_size)))
        labels.append(folders.index(folder))

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.3, random_state = 12)

X_train = np.array(X_train)
X_test = np.array(X_test)

X_train = X_train.reshape(X_train.shape[0], 60, 60, 1)
X_test = X_test.reshape(X_test.shape[0], 60, 60, 1)

In [6]:
# For sequential
def flatten(dimData, images):
    images = np.array(images)
    images = images.reshape(len(images), dimData)
    images = images.astype('float32')
    images /=255
    return images

#flatten data
dataDim = np.prod(images[0].shape)
train_data  = flatten(dataDim, X_train)
test_data = flatten(dataDim, X_test)

In [12]:
from keras.utils import to_categorical

#change labels to categorical
train_labels = np.array(y_train)
test_labels = np.array(y_test)
train_labels_one_hot = to_categorical(train_labels)
test_labels_one_hot = to_categorical(test_labels)

#determine the number of classes
classes = np.unique(train_labels)
nClasses  = len(classes)

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten

In [14]:
def get_model(model_name = 'sequential'):
    
    if model_name == 'CNN':
        # building a linear stack of layers with the sequential model
        model = Sequential()
        # convolutional layer
        model.add(Conv2D(25, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu', input_shape=(60,60,1)))
        model.add(MaxPool2D(pool_size=(1,1)))
        # flatten output of conv
        model.add(Flatten())
        # hidden layer
        model.add(Dense(100, activation='relu'))
        model.add(Dropout(0.1))
        # output layer
        model.add(Dense(nClasses, activation='softmax'))

        return model
    
    elif model_name == 'sequential':
        # dataDim = np.prod(images[0].shape)
        model = Sequential()
        model.add(Dense(256, activation = 'tanh', input_shape = (dataDim,)))
        model.add(Dropout(0.2))
        model.add(Dense(256, activation='tanh'))
        model.add(Dropout(0.2))
        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(nClasses, activation='softmax'))

        return model

In [15]:
model = get_model()

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_data, train_labels_one_hot, batch_size = 64, epochs=50, verbose=1,
                    validation_data=(test_data, test_labels_one_hot))

Train on 5239 samples, validate on 2246 samples
Epoch 1/50
5239/5239 [==============================] - 4s 858us/step - loss: 0.7907 - acc: 0.4921 - val_loss: 0.7025 - val_acc: 0.4951
Epoch 2/50
5239/5239 [==============================] - 4s 750us/step - loss: 0.7060 - acc: 0.4965 - val_loss: 0.6943 - val_acc: 0.5049
Epoch 3/50
5239/5239 [==============================] - 4s 744us/step - loss: 0.6961 - acc: 0.5163 - val_loss: 0.8924 - val_acc: 0.5049
Epoch 4/50
5239/5239 [==============================] - 3s 638us/step - loss: 0.3410 - acc: 0.8601 - val_loss: 6.3205e-04 - val_acc: 1.0000
Epoch 5/50
5239/5239 [==============================] - 4s 707us/step - loss: 0.1064 - acc: 0.9817 - val_loss: 1.6101e-05 - val_acc: 1.0000
Epoch 6/50
5239/5239 [==============================] - 3s 579us/step - loss: 1.2805e-04 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 7/50
5239/5239 [==============================] - 3s 548us/step - loss: 0.3857 - acc: 0.9710 - val_loss: 1.1843e-0

In [21]:
#save model
model.save(r'c:\Users\neera\Desktop\Lucid_demo'+'/model_square_tri.h5')

In [23]:
from keras.models import load_model

model = load_model(r"C:\Users\neera\Desktop\Lucid_demo\model_square_tri.h5")

AttributeError: 'str' object has no attribute 'decode'

In [19]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

ModuleNotFoundError: No module named 'tensorflow.compat'

In [17]:
model.load_graphdef()

AttributeError: 'Sequential' object has no attribute 'load_graphdef'